# Notebook with preprocessing collected data for ML

In [1]:
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

import babel.numbers

import seaborn as sns

# USE THIS STYLE
# plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')
# 
# OR THIS STYLE
import aquarel

import warnings

warnings.filterwarnings("ignore")

theme = aquarel.load_theme("arctic_light")
theme.set_font(family="serif")
theme.apply()

# Сделаем автоподгрузку всех изменений при перепрогонке ячейки
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
custom_pallete = {
    "red": "#BF616A",
    "orange": "#D08770",
    "yellow": "#EBCB8B",
    "green": "#A3BE8C",
    "purple": "#B48EAD",
    "light_green": "#8FBCBB",
    "light_gray_blue": "#88C0D0",
    "light_blue": "#81A1C1",
    "blue": "#5E81AC",
}

## Load data

In [3]:
df_cian = pd.read_csv("./data/cian_houses_dataset.csv")
df_cian.head()

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
0,18374400,"Москва, СВАО, р-н Марьина роща, Тэйт жилой ком...",55.802530,37.620945,Марьина Роща,34,49,38.28,11.8,unknown,...,3.12,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/302789697/,1
1,49954800,"Москва, ЦАО, р-н Таганский, Славянская пл., 2/5с1",55.752541,37.635727,Китай-город,7,8,53.20,unknown,24.1,...,3.0,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/301496936/,1
2,8170000,"Москва, НАО (Новомосковский), Филимонковское п...",55.552637,37.337172,Аэропорт Внуково,2,5,31.00,unknown,10.0,...,3.0,Во двор,unknown,Нет информации,Нет,unknown,Вторичка,unknown,https://www.cian.ru/sale/flat/303552909/,1
3,20206500,"Москва, СВАО, р-н Останкинский, ул. Годовикова...",55.810466,37.624247,Алексеевская,34,37,28.50,21.4,unknown,...,3.01,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/306568424/,1
4,17119620,"Москва, ЮАО, р-н Даниловский, Автозаводская ул...",55.706597,37.632285,Тульская,6,20,38.82,10.6,18.6,...,2.85,Во двор,Монолитный,unknown,unknown,unknown,Новостройка,unknown,https://www.cian.ru/sale/flat/305679800/,1


In [4]:
df_cian.shape

(8053, 22)

Разделим на численные и категориальные фитчи

In [5]:
numerical_features = [f for f in df_cian.columns if df_cian.dtypes[f] != "object"]
categorical_features = [f for f in df_cian.columns if df_cian.dtypes[f] == "object"]

In [6]:
numerical_features

['price', 'geo_lat', 'geo_lng', 'floor', 'floor_count', 'square', 'room_count']

In [7]:
df_cian["price"].describe()

count    8.053000e+03
mean     1.169362e+08
std      2.377114e+08
min      9.000000e+05
25%      1.496352e+07
50%      3.938630e+07
75%      1.150000e+08
max      3.592900e+09
Name: price, dtype: float64

## Уберем выбросы 

In [8]:
df_cian["room_count"].value_counts(), df_cian.shape

(room_count
 3    1462
 2    1435
 1    1425
 4    1380
 5    1343
 6    1008
 Name: count, dtype: int64,
 (8053, 22))

In [9]:
df_tmp = df_cian[df_cian["room_count"] == 1]

df_tmp = df_tmp[
    df_tmp["price"].between(
        df_tmp["price"].quantile(0.05), df_tmp["price"].quantile(0.95)
    )
]
df_tmp = df_tmp[
    df_tmp["square"].between(
        df_tmp["square"].quantile(0.05), df_tmp["square"].quantile(0.95)
    )
]

for i in range(2, 6 + 1):
    df_tmp_i = df_cian[df_cian["room_count"] == i]

    df_tmp_i = df_tmp_i[
        df_tmp_i["price"].between(
            df_tmp_i["price"].quantile(0.05), df_tmp_i["price"].quantile(0.95)
        )
    ]
    df_tmp_i = df_tmp_i[
        df_tmp_i["square"].between(
            df_tmp_i["square"].quantile(0.05), df_tmp_i["square"].quantile(0.95)
        )
    ]

    df_tmp = pd.concat([df_tmp, df_tmp_i], ignore_index=True, sort=False)

df_cian = df_tmp

In [10]:
df_cian["room_count"].value_counts(), df_cian.shape

(room_count
 3    1184
 2    1166
 1    1153
 4    1119
 5    1087
 6     815
 Name: count, dtype: int64,
 (6524, 22))

## Заполним пропуски в данных с площадями и высотой потолков

In [11]:
df_cian["square"] = df_cian["square"].apply(float)

Логика заполнения: получить соотношение жилой площади к общей, опираясь на количество комнат в квартире

In [12]:
tmp = df_cian[df_cian["living_square"] != "unknown"][["square", "living_square", "room_count"]]
tmp["lsqr/sqr"] = tmp["living_square"].apply(float) / tmp["square"]
tmp = tmp.groupby(by="room_count", as_index=False).agg({"lsqr/sqr": "mean"})
tmp

,room_count,lsqr/sqr
0,1,0.471396
1,2,0.546163
2,3,0.584735
3,4,0.586465
4,5,0.625507
5,6,0.645301


In [13]:
tmp = df_cian.merge(tmp, on="room_count")
tmp.loc[tmp["living_square"] == "unknown", "living_square"] = tmp["square"] * tmp["lsqr/sqr"]
tmp["living_square"] = tmp["living_square"].apply(lambda x: round(float(x), 2))
df_cian["living_square"] = tmp["living_square"]

Логика заполнения: получить соотношение площади кухни к общей, опираясь на количество комнат в квартире

In [14]:
tmp = df_cian[df_cian["kitchen_square"] != "unknown"][["square", "kitchen_square", "room_count"]]
tmp["ksqr/sqr"] = tmp["kitchen_square"].apply(float) / tmp["square"]
tmp = tmp.groupby(by="room_count", as_index=False).agg({"ksqr/sqr": "mean"})
tmp

,room_count,ksqr/sqr
0,1,0.267350
1,2,0.215836
2,3,0.175731
3,4,0.162067
4,5,0.121519
5,6,0.097665


In [15]:
tmp = df_cian.merge(tmp, on="room_count")
tmp.loc[tmp["kitchen_square"] == "unknown", "kitchen_square"] = tmp["square"] * tmp["ksqr/sqr"]
tmp["kitchen_square"] = tmp["kitchen_square"].apply(lambda x: round(float(x), 2))
df_cian["kitchen_square"] = tmp["kitchen_square"]

Логика заполнения: средняя высота потолков по всем данным

In [16]:
df_cian.loc[df_cian["ceiling_height"] == "unknown", "ceiling_height"] = np.mean(
    df_cian[df_cian["ceiling_height"] != "unknown"]["ceiling_height"]
    .apply(lambda x: round(float(x), 2))
    .values
)
df_cian["ceiling_height"] = df_cian["ceiling_height"].apply(lambda x: round(float(x), 2))

In [17]:
df_cian.head()

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
0,18374400,"Москва, СВАО, р-н Марьина роща, Тэйт жилой ком...",55.802530,37.620945,Марьина Роща,34,49,38.28,11.80,10.23,...,3.12,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/302789697/,1
1,8170000,"Москва, НАО (Новомосковский), Филимонковское п...",55.552637,37.337172,Аэропорт Внуково,2,5,31.00,14.61,10.00,...,3.00,Во двор,unknown,Нет информации,Нет,unknown,Вторичка,unknown,https://www.cian.ru/sale/flat/303552909/,1
2,20206500,"Москва, СВАО, р-н Останкинский, ул. Годовикова...",55.810466,37.624247,Алексеевская,34,37,28.50,21.40,7.62,...,3.01,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/306568424/,1
3,17119620,"Москва, ЮАО, р-н Даниловский, Автозаводская ул...",55.706597,37.632285,Тульская,6,20,38.82,10.60,18.60,...,2.85,Во двор,Монолитный,unknown,unknown,unknown,Новостройка,unknown,https://www.cian.ru/sale/flat/305679800/,1
4,7550000,"Москва, НАО (Новомосковский), Филимонковское п...",55.551456,37.339499,Филатов луг,2,5,30.00,15.00,9.00,...,2.80,Во двор,unknown,Нет информации,Нет,unknown,Вторичка,unknown,https://www.cian.ru/sale/flat/306451520/,1


In [18]:
numerical_features = [f for f in df_cian.columns if df_cian.dtypes[f] != "object"]
categorical_features = [f for f in df_cian.columns if df_cian.dtypes[f] == "object"]
numerical_features, categorical_features

(['price',
  'geo_lat',
  'geo_lng',
  'floor',
  'floor_count',
  'square',
  'living_square',
  'kitchen_square',
  'ceiling_height',
  'room_count'],
 ['location',
  'metro',
  'year',
  'finish_type',
  'view',
  'house_type',
  'heating',
  'breakdown',
  'parking',
  'accomodation_type',
  'author',
  'url'])

## Год постройки

In [19]:
df_cian["year"].unique()

array(['2027', '2021', '2024', '2013', '2026', 'unknown', '2011', '2023',
       '1967', '2019', '1978', '1981', '2008', '2004', '2017', '1960',
       '1959', '2022', '2007', '1989', '2006', '2025', '2018', '1984',
       '1973', '1997', '2016', '2009', '2028', '2015', '1966', '1988',
       '1971', '1972', '1998', '1970', '1986', '2014', '1977', '1980',
       '2020', '1985', '1968', '1999', '1938', '1975', '2012', '1969',
       '1963', '2003', '1983', '1964', '1965', '1961', '1974', '2010',
       '2005', '1976', '1987', '1996', '1991', '1995', '1979', '2002',
       '2001', '1962', '1956', '1958', '1982', '1950', '1993', '1990',
       '2000', '1949', '1902', '1931', '1939', '1957', '1897', '1933',
       '1955', '1917', '1926', '1952', '1953', '1914', '1941', '1895',
       '1992', '1994', '1935', '1876', '1928', '1954', '1934', '1890',
       '1912', '1906', '1910', '1940', '1908', '1913', '1886', '1937',
       '1927', '1901', '1925', '1904', '1893', '1900', '1943', '1903',
   

Год &mdash; это числовые данные, но присутсвуют неизвестные значения.
Заменим их на year = -1000

In [20]:
def year_transform(year):
    if type(year) != str:
        year = str(year)
    x = re.findall(r"\d+", year)
    if len(x) == 0:
        return "-1000"
    else:
        if len(x[0]) != 4:
            return "-1000"
        elif x[0][0] not in "12":
            return "-1000"
        else:
            return x[0]


In [21]:
df_cian["year"] = df_cian["year"].apply(lambda x: int(year_transform(x)))
df_cian[df_cian["year"] == -1000].head()

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
8,10800000,"Москва, ЮАО, р-н Даниловский, Автозаводская ул...",55.702313,37.640632,Автозаводская,3,4,45.0,21.21,10.0,...,3.1,Во двор,unknown,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/303618522/,1
195,18000000,"Москва, СЗАО, р-н Щукино, Большой Волоколамски...",55.808457,37.489014,Войковская,19,22,43.0,22.00,14.0,...,3.0,unknown,unknown,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/303573991/,1
220,18500000,"Москва, СЗАО, р-н Щукино, Большой Волоколамски...",55.808457,37.489014,Щукинская,19,22,43.0,22.00,14.0,...,3.0,unknown,Монолитный,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/305393056/,1
383,9900000,"Москва, СЗАО, р-н Хорошево-Мневники, Живописна...",55.783220,37.453414,Народное Ополчение,3,4,31.8,15.50,13.3,...,3.1,Во двор,Панельный,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/305815291/,1
452,13800000,"Москва, САО, р-н Головинский, Михалковская ул....",55.839886,37.512038,Коптево,10,10,35.3,12.00,9.0,...,3.1,unknown,unknown,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/305333395/,1


In [22]:
numerical_features = [f for f in df_cian.columns if df_cian.dtypes[f] != "object"]
categorical_features = [f for f in df_cian.columns if df_cian.dtypes[f] == "object"]
numerical_features, categorical_features

(['price',
  'geo_lat',
  'geo_lng',
  'floor',
  'floor_count',
  'square',
  'living_square',
  'kitchen_square',
  'year',
  'ceiling_height',
  'room_count'],
 ['location',
  'metro',
  'finish_type',
  'view',
  'house_type',
  'heating',
  'breakdown',
  'parking',
  'accomodation_type',
  'author',
  'url'])

## Исключение столбцов

Никак не пригодятся столбцы parking, location, url

In [23]:
df_cian = df_cian.drop(columns=["location", "parking", "url"])

df_cian.head()

,price,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,year,finish_type,ceiling_height,view,house_type,heating,breakdown,accomodation_type,author,room_count
0,18374400,55.802530,37.620945,Марьина Роща,34,49,38.28,11.80,10.23,2027,Предчистовая,3.12,unknown,Монолитно-кирпичный,unknown,unknown,Новостройка,КОРТРОС,1
1,8170000,55.552637,37.337172,Аэропорт Внуково,2,5,31.00,14.61,10.00,2021,Дизайнерский,3.00,Во двор,unknown,Нет информации,Нет,Вторичка,unknown,1
2,20206500,55.810466,37.624247,Алексеевская,34,37,28.50,21.40,7.62,2024,"Без отделки, предчистовая, чистовая",3.01,На улицу,Монолитно-кирпичный,unknown,unknown,Новостройка,КОРТРОС,1
3,17119620,55.706597,37.632285,Тульская,6,20,38.82,10.60,18.60,2024,unknown,2.85,Во двор,Монолитный,unknown,unknown,Новостройка,unknown,1
4,7550000,55.551456,37.339499,Филатов луг,2,5,30.00,15.00,9.00,2021,Дизайнерский,2.80,Во двор,unknown,Нет информации,Нет,Вторичка,unknown,1


In [24]:
numerical_features = [f for f in df_cian.columns if df_cian.dtypes[f] != "object"]
categorical_features = [f for f in df_cian.columns if df_cian.dtypes[f] == "object"]
numerical_features, categorical_features

(['price',
  'geo_lat',
  'geo_lng',
  'floor',
  'floor_count',
  'square',
  'living_square',
  'kitchen_square',
  'year',
  'ceiling_height',
  'room_count'],
 ['metro',
  'finish_type',
  'view',
  'house_type',
  'heating',
  'breakdown',
  'accomodation_type',
  'author'])

## Чистка столбцов

### finish_type

In [25]:
def finish_type_transform(expr: str):

    expr = expr.lower()

    # Changed categories

    res = re.findall(r"без отделки", expr)
    if len(res) != 0:
        return "Без отделки"
    
    res = re.findall(r"черновая", expr)
    if len(res) != 0:
        return "Черновая"
    
    res = re.findall(r"предчистовая", expr)
    if len(res) != 0:
        return "Предчистовая"
    
    res = re.findall(r"чистовая", expr)
    if len(res) != 0:
        return "Чистовая"
    
    res = re.findall(r"с отделкой", expr)
    if len(res) != 0:
        return "Без ремонта"
    

    # Unchanged categories

    res = re.findall(r"без ремонта", expr)
    if len(res) != 0:
        return "Без ремонта"
    
    res = re.findall(r"дизайнерский", expr)
    if len(res) != 0:
        return "Дизайнерский"

    res = re.findall(r"евроремонт", expr)
    if len(res) != 0:
        return "Евроремонт"
    
    res = re.findall(r"косметический", expr)
    if len(res) != 0:
        return "Косметический"
    
    
    # Else

    return "unknown"


In [26]:
df_cian["finish_type"] = df_cian["finish_type"].apply(finish_type_transform)

In [27]:
df_cian["finish_type"].value_counts()

finish_type
Дизайнерский     1698
Без отделки      1365
Без ремонта      1224
Евроремонт        857
Косметический     563
unknown           400
Предчистовая      200
Чистовая          178
Черновая           39
Name: count, dtype: int64

for scaler

In [28]:
df_cian["finish_type"].unique()


array(['Предчистовая', 'Дизайнерский', 'Без отделки', 'unknown',
       'Косметический', 'Без ремонта', 'Чистовая', 'Евроремонт',
       'Черновая'], dtype=object)

### house_type

In [29]:
def house_type_transform(expr: str):

    expr = expr.lower()

    # Changed categories

    res = re.findall(r"монолитно-кирпичный", expr)
    if len(res) != 0:
        return "Монолитно-кирпичный"
    
    res = re.findall(r"газобетонный блок", expr)
    if len(res) != 0:
        return "Блочный"
    
    res = re.findall(r"газобетон", expr)
    if len(res) != 0:
        return "Блочный"
    
    res = re.findall(r"деревянный", expr)
    if len(res) != 0:
        return "unknown"

    # Unchanged categories

    res = re.findall(r"монолитный", expr)
    if len(res) != 0:
        return "Монолитный"
    
    res = re.findall(r"блочный", expr)
    if len(res) != 0:
        return "Блочный"
    
    res = re.findall(r"кирпичный", expr)
    if len(res) != 0:
        return "Кирпичный"

    res = re.findall(r"панельный", expr)
    if len(res) != 0:
        return "Панельный"
    
    res = re.findall(r"сталинский", expr)
    if len(res) != 0:
        return "Сталинский"
    
    
    # Else

    return "unknown"


In [30]:
df_cian["house_type"] = df_cian["house_type"].apply(house_type_transform)

In [31]:
df_cian["house_type"].value_counts()


house_type
Монолитный             2647
Монолитно-кирпичный    1222
unknown                1055
Кирпичный               994
Панельный               455
Блочный                 147
Сталинский                4
Name: count, dtype: int64

for scaler

In [32]:
df_cian["house_type"].unique()

array(['Монолитно-кирпичный', 'unknown', 'Монолитный', 'Панельный',
       'Блочный', 'Кирпичный', 'Сталинский'], dtype=object)

### heating

In [33]:
def heating_transform(expr: str):
    
    expr = expr.lower()

    # Changed categories

    res = re.findall(r"нет информации", expr)
    if len(res) != 0:
        return "unknown"
    
    res = re.findall(r"котел\/квартирное отопление", expr)
    if len(res) != 0:
        return "Индивидуальный тепловой пункт"


    # Unchanged categories

    res = re.findall(r"центральное", expr)
    if len(res) != 0:
        return "Центральное"
    
    res = re.findall(r"индивидуальный тепловой пункт", expr)
    if len(res) != 0:
        return "Индивидуальный тепловой пункт"
    
    res = re.findall(r"автономная котельная", expr)
    if len(res) != 0:
        return "Автономная котельная"

    res = re.findall(r"электрическое", expr)
    if len(res) != 0:
        return "Электрическое"
    
    res = re.findall(r"нет", expr)
    if len(res) != 0:
        return "Нет"


    # Else

    return "unknown"


In [34]:
df_cian["heating"] = df_cian["heating"].apply(heating_transform)


In [35]:
df_cian["heating"].value_counts()


heating
unknown                          3286
Центральное                      2544
Индивидуальный тепловой пункт     573
Автономная котельная              113
Электрическое                       7
Нет                                 1
Name: count, dtype: int64

for scaler

In [36]:
df_cian["heating"].unique()

array(['unknown', 'Индивидуальный тепловой пункт', 'Автономная котельная',
       'Центральное', 'Электрическое', 'Нет'], dtype=object)

### author

In [37]:
authors = (
    df_cian["author"]
    .value_counts()
    .to_frame()
    .reset_index()
    .sort_values(by="count", ascending=False)
).iloc[:40]
display(authors)
authors = authors["author"].values
authors

,author,count
0,unknown,4999
1,Группа Самолет,225
2,А101,123
3,MR Group,75
4,ГК ФСК,67
5,Класс,56
6,КОРТРОС,55
7,ГК Инград,54
8,Level Group,51
9,TEKTA GROUP,46


array(['unknown', 'Группа Самолет', 'А101', 'MR Group', 'ГК ФСК', 'Класс',
       'КОРТРОС', 'ГК Инград', 'Level Group', 'TEKTA GROUP', '1-й ДСК',
       'ДОНСТРОЙ', 'ГК Основа', 'Optima Development', 'Центр-Инвест',
       'Sminex', 'Capital Group', 'Главстрой', 'Абсолют Недвижимость',
       'Концерн КРОСТ', 'Рублево-Архангельское', 'FORMA', 'Группа Эталон',
       'Группа Родина', 'ANT Development', 'ГК ТехноСтройОлимп',
       'СЗ Киноцентр', 'СЗ Риверхаус', 'РГ-Девелопмент',
       'Компания PIONEER', 'STONE', 'ПИК', 'Regions Development',
       'AUTHOR DEVELOPMENT', 'ГК Сумма Элементов', 'AFI Development',
       'Ташир', 'СЗ Минские холмы', 'DOGMA', 'ГК Страна Девелопмент'],
      dtype=object)

In [38]:
def author_transform(expr):
    if expr not in authors:
        return "unknown"
    return expr

In [39]:
df_cian["author"] = df_cian["author"].apply(author_transform)

In [40]:
df_cian["author"].value_counts()


author
unknown                  5243
Группа Самолет            225
А101                      123
MR Group                   75
ГК ФСК                     67
Класс                      56
КОРТРОС                    55
ГК Инград                  54
Level Group                51
TEKTA GROUP                46
1-й ДСК                    44
ДОНСТРОЙ                   42
ГК Основа                  38
Optima Development         28
Центр-Инвест               24
Sminex                     22
Capital Group              20
Главстрой                  19
Абсолют Недвижимость       18
Концерн КРОСТ              17
FORMA                      17
Рублево-Архангельское      17
Группа Эталон              16
Группа Родина              15
СЗ Киноцентр               14
ГК ТехноСтройОлимп         14
ANT Development            14
СЗ Риверхаус               14
Компания PIONEER           13
РГ-Девелопмент             13
ПИК                        12
AUTHOR DEVELOPMENT         12
Regions Development        12
STO

for scaler

In [41]:
df_cian["author"].unique()

array(['КОРТРОС', 'unknown', 'MR Group', 'Главстрой',
       'Рублево-Архангельское', 'TEKTA GROUP', 'Компания PIONEER',
       'FORMA', 'Level Group', 'ДОНСТРОЙ', 'Концерн КРОСТ', 'Класс',
       'Regions Development', 'РГ-Девелопмент', 'Sminex', 'Группа Родина',
       'А101', 'ГК ФСК', 'Абсолют Недвижимость', '1-й ДСК', 'ГК Основа',
       'Центр-Инвест', 'DOGMA', 'Группа Самолет', 'ГК ТехноСтройОлимп',
       'AUTHOR DEVELOPMENT', 'ПИК', 'ГК Инград', 'ГК Сумма Элементов',
       'СЗ Киноцентр', 'AFI Development', 'Ташир', 'ANT Development',
       'Optima Development', 'Capital Group', 'СЗ Минские холмы',
       'Группа Эталон', 'СЗ Риверхаус', 'ГК Страна Девелопмент', 'STONE'],
      dtype=object)

## New features

In [42]:
df_cian["floor_ratio"] = df_cian["floor"] / df_cian["floor_count"]

df_cian["room_ratio"] = df_cian["square"] / df_cian["room_count"]

df_cian["living_ratio"] = df_cian["living_square"] / df_cian["square"]
df_cian["kitchen_ratio"] = df_cian["kitchen_square"] / df_cian["square"]

In [43]:
df_cian

,price,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,year,...,house_type,heating,breakdown,accomodation_type,author,room_count,floor_ratio,room_ratio,living_ratio,kitchen_ratio
0,18374400,55.802530,37.620945,Марьина Роща,34,49,38.28,11.80,10.23,2027,...,Монолитно-кирпичный,unknown,unknown,Новостройка,КОРТРОС,1,0.693878,38.280000,0.308255,0.267241
1,8170000,55.552637,37.337172,Аэропорт Внуково,2,5,31.00,14.61,10.00,2021,...,unknown,unknown,Нет,Вторичка,unknown,1,0.400000,31.000000,0.471290,0.322581
2,20206500,55.810466,37.624247,Алексеевская,34,37,28.50,21.40,7.62,2024,...,Монолитно-кирпичный,unknown,unknown,Новостройка,КОРТРОС,1,0.918919,28.500000,0.750877,0.267368
3,17119620,55.706597,37.632285,Тульская,6,20,38.82,10.60,18.60,2024,...,Монолитный,unknown,unknown,Новостройка,unknown,1,0.300000,38.820000,0.273055,0.479134
4,7550000,55.551456,37.339499,Филатов луг,2,5,30.00,15.00,9.00,2021,...,unknown,unknown,Нет,Вторичка,unknown,1,0.400000,30.000000,0.500000,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6519,1000000000,55.717503,37.504726,Минская,2,7,481.00,310.39,35.00,2019,...,Монолитный,Центральное,Нет,Вторичка,unknown,6,0.285714,80.166667,0.645301,0.072765
6520,1100000000,55.746963,37.614949,Боровицкая,4,7,508.00,327.81,49.61,2021,...,Монолитный,Центральное,Нет,Вторичка / Пентхаус,unknown,6,0.571429,84.666667,0.645295,0.097657
6521,270000000,55.735525,37.453639,Кунцевская,22,22,580.00,450.00,100.00,2011,...,Панельный,Центральное,Нет,Вторичка,unknown,6,1.000000,96.666667,0.775862,0.172414
6522,987184440,55.756153,37.606640,Арбатская,3,5,235.40,122.00,16.80,2023,...,Монолитный,unknown,unknown,Новостройка,unknown,6,0.600000,39.233333,0.518267,0.071368


In [44]:
numerical_features = [f for f in df_cian.columns if df_cian.dtypes[f] != "object"]
categorical_features = [f for f in df_cian.columns if df_cian.dtypes[f] == "object"]
numerical_features, categorical_features

(['price',
  'geo_lat',
  'geo_lng',
  'floor',
  'floor_count',
  'square',
  'living_square',
  'kitchen_square',
  'year',
  'ceiling_height',
  'room_count',
  'floor_ratio',
  'room_ratio',
  'living_ratio',
  'kitchen_ratio'],
 ['metro',
  'finish_type',
  'view',
  'house_type',
  'heating',
  'breakdown',
  'accomodation_type',
  'author'])

## Export training dataset

In [45]:
df_cian.to_csv("./data/cian_houses_training.csv", index=False)

## P.S.

Пункты for scaler пригодятся при построении интерфейса для взаимодействия с программой:
условно, если пользователь введелет что-то не то, или даст данные, на которых модель не может отработать корректно, то выскочит предупреждение или программа отнесет данные к типу "unknown".